# User Interface

In [9]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from xgboost import XGBClassifier
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import ipywidgets as widgets
from IPython.display import display
import joblib
import pickle

In [10]:
with open("known_song_titles.pkl", "rb") as f:
    known_song_titles = pickle.load(f)

with open("known_artists.pkl", "rb") as f:
    known_artists = pickle.load(f)

In [11]:

# Load the fitted objects and model
le_song_title = joblib.load('label_encoder_song_title.pkl')
le_artist = joblib.load('label_encoder_artist.pkl')
scaler = joblib.load('standard_scaler.pkl')
xgb = joblib.load('xgb_Model.pkl')

# Spotify API setup
client_id = '6291dd76ffb744299366b81b9cddfe5c'
client_secret = '14e1d9b1399a4d8e8ea34e09717b4815'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Input Widget for Spotify track URL
spotify_url = widgets.Text(description='Spotify URL:')

# Action Button
btn_predict = widgets.Button(description='Predict')

# Output Area
output = widgets.Output()

# Button click event
def on_button_click(b):
    with output:
        output.clear_output()
        
        # Extract track ID from Spotify URL
        track_id = spotify_url.value.split('/')[-1].split('?')[0]
        
        # Get track features and info
        track = sp.audio_features(track_id)[0]
        track_info = sp.track(track_id)

        # Create DataFrame from track features
        df = pd.DataFrame([{
            'danceability': track['danceability'],
            'duration_ms': track['duration_ms'],
            'energy': track['energy'],
            'instrumentalness': track['instrumentalness'],
            'key': track['key'],
            'liveness': track['liveness'],
            'mode': track['mode'],
            'speechiness': track['speechiness'],
            'tempo': track['tempo'],
            'time_signature': track['time_signature'],
            'song_title': track_info['name'],
            'artist': track_info['artists'][0]['name']
        }])

        # Feature selection
        df.drop(['acousticness', 'loudness', 'valence'], axis=1, inplace=True, errors='ignore')

        # Handle unknown labels by assigning them to a special "Unknown" category
        df['song_title'] = df['song_title'].apply(lambda x: x if x in known_song_titles else 'Unknown')
        df['artist'] = df['artist'].apply(lambda x: x if x in known_artists else 'Unknown')

        # Label encoding for 'song_title' and 'artist'
        df['song_title'] = le_song_title.transform(df['song_title'])
        df['artist'] = le_artist.transform(df['artist'])

        # Scaling
        columns_to_standardize = ['instrumentalness', 'danceability', 'duration_ms', 'energy', 'speechiness', 'liveness', 'tempo']
        df[columns_to_standardize] = scaler.transform(df[columns_to_standardize])

        # Make prediction
        prediction = xgb.predict(df)

        # Display the prediction and track info
        print(f"Title: {track_info['name']}")
        print(f"Artist: {track_info['artists'][0]['name']}")
        
        # Display Post-Processing Features
        print("Post-Processed Features:")
        display(df)  # Using IPython's display function to pretty-print the DataFrame

        # Display Prediction with Visual Indication
        if prediction == 1:
            print("\033[92m Predicted Result: 1 \033[0m")  # Green for "1"
        else:
            print("\033[91m Predicted Result: 0 \033[0m")  # Red for "0"

# Bind button click event to function
btn_predict.on_click(on_button_click)

# Display widgets
display(spotify_url, btn_predict, output)

Text(value='', description='Spotify URL:')

Button(description='Predict', style=ButtonStyle())

Output()

In [13]:
#pip install streamlit